In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
import time

pd.set_option('display.max_columns', 50)

from evaluation_and_testing.testing import evaluate_train_test_split_explicit
from evaluation_and_testing.testing import evaluate_leave_one_out_explicit
from evaluation_and_testing.testing import evaluate_train_test_split_implicit
from evaluation_and_testing.testing import evaluate_leave_one_out_implicit

from recommenders.recommender import Recommender

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the data

In [2]:
ml_ratings_df = pd.read_csv(os.path.join("data", "movielens_small", "ratings.csv")).rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
ml_movies_df = pd.read_csv(os.path.join("data", "movielens_small", "movies.csv")).rename(columns={'movieId': 'item_id'})
ml_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')

display(ml_movies_df.head(10))

# Filter the data to reduce the number of movies
seed = 6789
rng = np.random.RandomState(seed=seed)
left_ids = rng.choice(ml_movies_df['item_id'], size=1000, replace=False)

ml_ratings_df = ml_ratings_df.loc[ml_ratings_df['item_id'].isin(left_ids)]
ml_movies_df = ml_movies_df.loc[ml_movies_df['item_id'].isin(left_ids)]
ml_df = ml_df.loc[ml_df['item_id'].isin(left_ids)]

print("Number of chosen interactions: {}".format(len(ml_ratings_df)))

display(ml_df.head(10))

,item_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Number of chosen interactions: 9692


,user_id,item_id,rating,timestamp,title,genres
572,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
573,5,50,4.0,847434881,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
574,6,50,1.0,845553381,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
575,7,50,4.5,1106635993,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
576,8,50,5.0,839463644,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
577,16,50,4.0,1377476781,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
578,17,50,4.5,1305697013,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
579,18,50,5.0,1455049343,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
580,23,50,4.0,1107163741,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
581,24,50,4.0,1458942023,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


# Baseline recommenders

**Task 1.** Implement the MostPopularRecommender (check the slides for class 1), evaluate it with leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [ ]:
class MostPopularRecommender(Recommender):
    """
    Base recommender class.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        # Write your code here
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        # Write your code here
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Write your code here
        
    
# Quick test of the recommender

most_popular_recommender = MostPopularRecommender()
most_popular_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = most_popular_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(HTML(recommendations.to_html()))

most_popular_recommender = MostPopularRecommender()

most_popular_results = [['MostPopularRecommender'] + list(evaluate_leave_one_out_implicit(
    most_popular_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df, max_evals=300, seed=6789))]

most_popular_results = pd.DataFrame(
    most_popular_results, 
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(most_popular_results.to_html()))

**Task 2.** Implement the HighestRatedRecommender (check the slides for class 1), but recommend only those movies which got at least 50 ratings. Evaluate it with leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [23]:
class HighestRatedRecommender(Recommender):
    """
    Base recommender class.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        self.model = None
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df['popularity'] = 1
        movie_popularity = interactions_df.loc[:, ['item_id', 'rating', 'popularity']].groupby('item_id').agg({'rating': 'mean', 'popularity': 'sum'}).sort_values(by='rating', ascending=False)
        movie_popularity = movie_popularity.loc[movie_popularity['popularity'] >= 50].reset_index()
        self.model = movie_popularity
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        display(self.model)
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        user_recommendations = []
        
        for idx, user in users_df.iterrows():
            for i in range(n_recommendations):
                user_recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': self.model.iloc[i]['item_id'],
                        'score': self.model.iloc[i]['rating']
                    }
                )

            
        user_recommendations = pd.DataFrame(user_recommendations)
        recommendations = pd.concat([recommendations, user_recommendations])
        return recommendations
    
# # Quick test of the recommender

highest_rated_recommender = HighestRatedRecommender()

highest_rated_recommender.fit(ml_ratings_df, None, None)

print(highest_rated_recommender.recommend(pd.DataFrame([[1], [3]], columns=['user_id']), ml_movies_df, n_recommendations=5)[['user_id', 'item_id']])

# highest_rated_recommender = HighestRatedRecommender()
# highest_rated_recommender.fit(ml_ratings_df, None, ml_movies_df)
# recommendations = highest_rated_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)

# recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
# print("Recommendations")
# display(HTML(recommendations.to_html()))

# highest_rated_recommender = HighestRatedRecommender()

# highest_rated_results = [['HighestRatedRecommender'] + list(evaluate_leave_one_out_implicit(
#     highest_rated_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, max_evals=300, seed=6789))]

# highest_rated_results = pd.DataFrame(
#     highest_rated_results, 
#     columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

# display(HTML(highest_rated_results.to_html()))

,item_id,rating,popularity
0,858,4.289062,192
1,750,4.268041,97
2,1213,4.250000,126
3,50,4.237745,204
4,1225,4.184211,76
5,1203,4.149123,57
6,2502,4.090426,94
7,1704,4.078014,141
8,2692,4.000000,75
9,6502,3.974138,58


  user_id  item_id
0       1    858.0
1       1    750.0
2       1   1213.0
3       1     50.0
4       1   1225.0
5       3    858.0
6       3    750.0
7       3   1213.0
8       3     50.0
9       3   1225.0


**Task 3.** Implement the RandomRecommender (check the slides for class 1), evaluate it with leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [12]:
class RandomRecommender(Recommender):
    """
    Base recommender class.
    """
    
    def __init__(self, seed=0):
        """
        Initialize base recommender params and variables.
        
        :param int seed: Seed for the random number generator.
        """
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
        self.model = None
        # Write your code here
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        movie_ids = list(items_df['item_id'].unique())
        self.model = movie_ids
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        user_recommendations = []
        
        for idx, user in users_df.iterrows():
            
            random_movie_ids = self.rng.choice(self.model, size=n_recommendations, replace=False)
            
            for idi in random_movie_ids:
                user_recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': idi,
                        'score': 1.0
                    }
                )

            
        user_recommendations = pd.DataFrame(user_recommendations)
        recommendations = pd.concat([recommendations, user_recommendations])
        return recommendations
    
# Quick test of the recommender

seed = 6789
random_recommender = RandomRecommender(seed=seed)

random_recommender.fit(ml_ratings_df, None, ml_movies_df)

print(random_recommender.recommend(pd.DataFrame([[1], [3]], columns=['user_id']), ml_movies_df, n_recommendations=5)[['user_id', 'item_id']])

# random_recommender = RandomRecommender(seed=6789)
# random_recommender.fit(ml_ratings_df, None, ml_movies_df)
# recommendations = random_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)

# recommendations = recommendations.merge(ml_movies_df, on='item_id', how='left')
# print("Recommendations")
# display(HTML(recommendations.to_html()))

# random_recommender = RandomRecommender(seed=seed)

# random_results = [['RandomRecommender'] + list(evaluate_leave_one_out_implicit(
#     random_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, max_evals=300, seed=seed))]

# random_results = pd.DataFrame(
#     random_results, 
#     columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

# display(HTML(random_results.to_html()))

  user_id item_id
0       1     472
1       1   26631
2       1   63276
3       1    2815
4       1   31553
5       3   66335
6       3   91974
7       3    3846
8       3    4397
9       3    2304


# Linear Regression Recommender

For every movie we transform its genres into one-hot encoded features and we normalize them, for every user we count percentages for all genres how often do they appear among films watched by the user, we multiply both vectors (for the item and the user) to obtain explanatory variables, and then we fit a linear regression model to those features and actual ratings.

**Task 4.** Implement the calculate_user_genres method for calculating a DataFrame with one row per user and columns corresponding to genres (e.g. 'user_action', 'user_drama') with values calculated as follows:

- count the number of times a given user watched a given genre,
- apply a natural logarithm to this value plus one,
- normalize those values so that the sum of them over all columns within a row is equal to 1.

Implement the calculate_item_genres method for replacing the 'genres' column with one column per genre (e.g. 'action', 'drama') with values calculated as follows:

- place 1 in every column for which the genre appears in genres,
- normalize those values so that the sum of them over all genre columns within a row is equal to 1.

If item_features is None, then first find all genres and prepare a list of them. If item_features is not None, then create columns based on this list. Return both the transformed DataFrame and the list of genres.  Do not use MultiLabelBinarizer.

Note that in this second method you have to preserve the remaining structure of the DataFrame.

Evaluate the LinearRegressionRecommender with it using leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [17]:
# Prepare data for tests
interactions_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("-", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(" ", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("(", "", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(")", "", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.lower()
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.split("|")

In [27]:
def calculate_user_genres(interactions_df):
    
    users_df = interactions_df.copy()[['user_id', 'genres']].explode('genres')
    users_df['val'] = 1
    users_df = pd.pivot_table(users_df, index='user_id', columns='genres', values='val', aggfunc='count', fill_value=0.0).add_prefix("user_")
    users_df.columns.name = None
    users_df = users_df.applymap(lambda x: np.log(x+1))
    users_df = users_df / users_df.sum(axis=1).values.reshape(-1, 1)
    
    return users_df

In [29]:
# Test the calculate_user_genres method
display(calculate_user_genres(interactions_df))
print(np.log(10+1))

,user_action,user_adventure,user_animation,user_children,user_comedy,user_crime,user_documentary,user_drama,user_fantasy,user_film_noir,user_horror,user_imax,user_musical,user_mystery,user_no_genres_listed,user_romance,user_sci_fi,user_thriller,user_war,user_western
user_id,,,,,,,,,,,,,,,,,,,,
1,10,8,3,5,8,6,0,4,3,0,0,0,1,1,0,5,4,6,2,1
2,2,0,0,0,0,2,0,6,0,0,0,0,0,0,0,1,1,3,0,1
3,2,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,2,1,0,0
4,1,1,2,3,11,5,0,15,2,1,0,1,3,0,0,7,0,2,1,1
5,0,0,1,1,1,1,0,3,1,0,0,1,1,1,0,3,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,11,9,2,5,37,15,1,67,10,1,3,2,1,5,0,28,7,15,5,1
607,7,3,0,1,6,4,0,10,1,0,3,0,1,1,0,2,4,3,1,0
608,28,13,5,15,43,18,0,25,13,0,12,2,2,4,0,10,18,23,2,0


,user_action,user_adventure,user_animation,user_children,user_comedy,user_crime,user_documentary,user_drama,user_fantasy,user_film_noir,user_horror,user_imax,user_musical,user_mystery,user_no_genres_listed,user_romance,user_sci_fi,user_thriller,user_war,user_western
user_id,,,,,,,,,,,,,,,,,,,,
1,0.102311,0.093749,0.059149,0.076449,0.093749,0.083027,0.000000,0.068670,0.059149,0.000000,0.000000,0.000000,0.029575,0.029575,0.0,0.076449,0.068670,0.083027,0.046875,0.029575
2,0.144386,0.000000,0.000000,0.000000,0.000000,0.144386,0.000000,0.255742,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.091097,0.091097,0.182194,0.000000,0.091097
3,0.256885,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.162077,0.000000,0.162077,0.000000,0.000000,0.000000,0.0,0.000000,0.256885,0.162077,0.000000,0.000000
4,0.035810,0.035810,0.056758,0.071621,0.128379,0.092569,0.000000,0.143242,0.056758,0.035810,0.000000,0.035810,0.071621,0.000000,0.0,0.107431,0.000000,0.056758,0.035810,0.035810
5,0.000000,0.000000,0.066667,0.066667,0.066667,0.066667,0.000000,0.133333,0.066667,0.000000,0.000000,0.066667,0.066667,0.066667,0.0,0.133333,0.000000,0.066667,0.066667,0.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.065798,0.060970,0.029090,0.047444,0.096320,0.073415,0.018354,0.111728,0.063494,0.018354,0.036708,0.029090,0.018354,0.047444,0.0,0.089163,0.055062,0.073415,0.047444,0.018354
607,0.113226,0.075484,0.000000,0.037742,0.105955,0.087634,0.000000,0.130566,0.037742,0.000000,0.075484,0.000000,0.037742,0.037742,0.0,0.059820,0.087634,0.075484,0.037742,0.000000
608,0.085929,0.067345,0.045723,0.070753,0.096567,0.075138,0.000000,0.083142,0.067345,0.000000,0.065454,0.028035,0.028035,0.041071,0.0,0.061191,0.075138,0.081099,0.028035,0.000000


2.3978952727983707


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

def calculate_item_genres(item_genres, item_features=None):
    # Write your code here
    
    return item_genres, item_features

In [ ]:
# Test the method

item_genres, item_features = calculate_item_genres(interactions_df)
display(item_genres)
print(item_features)

item_genres, item_features = calculate_item_genres(interactions_df, ['comedy', 'crime', 'drama', 'horror', 'test_category'])
display(item_genres)
print(item_features)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MultiLabelBinarizer

from recommenders.recommender import Recommender

class LinearRegressionRecommender(Recommender):
    """
    Linear regression recommender class.
    """
    
    def __init__(self):
        """
        Initialize recommender params and variables.
        """
        self.model = None
        self.mlb = None
        self.users_dict = None
        self.user_features = None
        
        self.uses_dot_product = True
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        # Transform genres to a more code-friendly form
        
        interactions_df = pd.merge(interactions_df, items_df, on='item_id')
        interactions_df = self._transform_genres(interactions_df)
        
        # Prepare user features
        
        users_df = interactions_df[['user_id', 'genres']].copy()
        users_df = users_df.explode('genres')
        users_df['val'] = 1
        users_df = users_df.pivot_table(index='user_id', columns='genres', values='val', aggfunc='count')
        users_df = users_df / users_df.sum(axis=1).values.reshape(-1, 1)
        users_df = users_df.rename_axis(None, axis=1).fillna(0)
        users_df = users_df.add_prefix('user_')
#         display(users_df.head(10))
        
        self.users_dict = users_df.to_dict('index')
        
        self.user_features = users_df.columns.tolist()
        
        interactions_df = interactions_df.merge(users_df, on='user_id')
#         display(interactions_df.head(10))
                
        # Prepare item features
        
        # Transform genres into binary values
        
        self.mlb = MultiLabelBinarizer()
        interactions_df = interactions_df.join(
            pd.DataFrame(self.mlb.fit_transform(interactions_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=interactions_df.index))
        
        # Normalize the values so that each movie's genres sum up to 1
        
        interactions_df[self.mlb.classes_] = interactions_df[self.mlb.classes_] \
            / interactions_df[self.mlb.classes_].sum(axis=1).values.reshape(-1, 1)
        
#         display(interactions_df.loc[:, self.mlb.classes_].head(10))

        # Prepare input data and fit the model
    
        # Dot product for personalization
        if self.uses_dot_product:
            interactions_df[self.mlb.classes_] = interactions_df[self.mlb.classes_] \
                * interactions_df[self.user_features].values
    
#         display(interactions_df.head(10))
        
        if self.uses_dot_product:  # Personalized
            x = interactions_df.loc[:, self.mlb.classes_].values
        else:  # Non-personalized
            x = interactions_df.loc[:, list(self.mlb.classes_) + self.user_features].values
        
        y = interactions_df['rating'].values
    
        self.model = LinearRegression().fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Transform the item to be scored into proper features
        
        items_df = items_df.copy()
        items_df = self._transform_genres(items_df)
        
        items_df = items_df.join(
            pd.DataFrame(self.mlb.transform(items_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=items_df.index))
        
        items_df[self.mlb.classes_] = items_df[self.mlb.classes_] \
            / items_df[self.mlb.classes_].sum(axis=1).values.reshape(-1, 1)

        # Score the item
    
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            if user['user_id'] in self.users_dict:
                user_df = pd.DataFrame.from_dict({user['user_id']: self.users_dict[user['user_id']]}, orient='index')
            else:
                user_df = pd.DataFrame.from_dict(
                    {user['user_id']: [1 / len(self.user_features)]*len(self.user_features)}, orient='index')
                user_df.columns = self.user_features
#             display(user_df)
#             display(items_df)
            input_df = items_df.copy()
            
            if self.uses_dot_product:
                input_df[self.mlb.classes_] = items_df[self.mlb.classes_] * user_df.values
#                 display(input_df)
                scores = self.model.predict(input_df.loc[:, self.mlb.classes_].values)
            else:
                input_df = input_df.merge(user_df, how='cross')
#                 display(input_df)
                scores = self.model.predict(input_df.loc[:, list(self.mlb.classes_) + self.user_features].values)
    
            chosen_pos = np.argsort(-scores)[:n_recommendations]
        
            user_recommendations = []
            for item_pos in chosen_pos:
                user_recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': input_df.iloc[item_pos]['item_id'],
                        'score': scores[item_pos]
                    }
                )
                
            user_recommendations = pd.DataFrame(user_recommendations)

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations
    
    def _transform_genres(self, df):
        """
        Transforms a string with genres into a list of cleaned genre names.
        
        :param pd.DataFrame df: A DataFrame with 'genres' column.
        """
        df.loc[:, 'genres'] = df['genres'].str.replace("-", "_", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace(" ", "_", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace("(", "", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace(")", "", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.lower()
        df.loc[:, 'genres'] = df['genres'].str.split("|")
        return df

In [ ]:
# Quick test of the recommender

lr_recommender = LinearRegressionRecommender()
lr_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = lr_recommender.recommend(pd.DataFrame([[3], [5], [39]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
display(recommendations)

### Train-test split test

In [ ]:
lr_recommender = LinearRegressionRecommender()

t0 = time.time()

results = [['LinearRegressionRecommender'] + list(evaluate_train_test_split_explicit(
    lr_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

### Leave-one-out test

In [ ]:
lr_recommender = LinearRegressionRecommender()

t0 = time.time()

results = [['LinearRegressionRecommender'] + list(evaluate_leave_one_out_explicit(
    lr_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

# TF-IDF Recommender
TF-IDF stands for term frequency–inverse document frequency. Typically Tf-IDF method is used to assign keywords (words describing the gist of a document) to documents in a corpus of documents.

In our case we will treat users as documents and genres as words.

Term-frequency is given by the following formula:
<center>
$$
    \text{tf}(g, u) = f_{g, u}
$$
</center>
where $f_{g, i}$ is the number of times genre $g$ appear for movies watched by user $u$.

Inverse document frequency is defined as follows:
<center>
$$
    \text{idf}(g) = \log \frac{N}{n_g}
$$
</center>
where $N$ is the number of users and $n_g$ is the number of users with $g$ in their genres list.

Finally, tf-idf is defined as follows:
<center>
$$
    \text{tfidf}(g, u) = \text{tf}(g, u) \cdot \text{idf}(g)
$$
</center>

In our case we will measure how often a given genre appears for movies watched by a given user vs how often it appears for all users. To obtain a movie score we will take the average of its genres' scores for this user.

**Task 5.** Implement the following method for calculating a TF-IDF scores in a form of a dict (use defaultdict):

`{(1, 'action'): 0.45306430692185395, (1, 'adventure'): 0.39370003643934415, (1, 'animation'): 0.20886242957049514, ...}`

without using TfidfVectorizer (you can use loops).

Evaluate the TFIDFRecommender with it using leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [ ]:
def calculate_tf_idf_scores(interactions_df):
    # Write your code here
    
    return tfidf_scores

In [ ]:
# Test the method
interactions_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("-", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(" ", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("(", "", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(")", "", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.lower()
        
tfidf_scores = calculate_tf_idf_scores(interactions_df)
print(tfidf_scores)

In [ ]:
class TFIDFRecommender(Recommender):
    """
    Recommender based on the TF-IDF method.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        self.tfidf_scores = None
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        self.tfidf_scores = defaultdict(lambda: 0.0)

        # Prepare the corpus for tfidf calculation
        
        interactions_df = pd.merge(interactions_df, items_df, on='item_id')
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("-", "_", regex=False)
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(" ", "_", regex=False)
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.lower()
                
        self.tfidf_scores = calculate_tf_idf_scores(interactions_df)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        # Transform genres to a unified form used by the vectorizer
        
        items_df = items_df.copy()
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace("-", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace(" ", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.lower()
        items_df.loc[:, 'genres'] = items_df['genres'].str.split("|")
                
        # Score items    
        
        for uix, user in users_df.iterrows():
            items = []
            for iix, item in items_df.iterrows():
                score = 0.0
                for genre in item['genres']:
                    score += self.tfidf_scores[(user['user_id'], genre)]
                score /= len(item['genres'])
                items.append((item['item_id'], score))
                
            items = sorted(items, key=lambda x: x[1], reverse=True)
            user_recommendations = pd.DataFrame({'user_id': user['user_id'],
                                                 'item_id': [item[0] for item in items][:n_recommendations],
                                                 'score': [item[1] for item in items][:n_recommendations]})

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations

In [ ]:
# Print movies watched by user 3, 5, 39

active_user_movies = ml_df.loc[(ml_df['user_id'] == 3) | (ml_df['user_id'] == 5) | (ml_df['user_id'] == 39)]
print("Active users history")
display(active_user_movies)

In [ ]:
# Quick test of the recommender

tfidf_recommender = TFIDFRecommender()
tfidf_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = tfidf_recommender.recommend(pd.DataFrame([[3], [5], [39]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(recommendations)

### Train-test split test

In [ ]:
tfidf_recommender = TFIDFRecommender()

t0 = time.time()

results = [['TFIDFRecommender'] + list(evaluate_train_test_split_implicit(
    tfidf_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

### Leave-one-out test

In [ ]:
tfidf_recommender = TFIDFRecommender()

t0 = time.time()

results = [['TFIDFRecommender'] + list(evaluate_leave_one_out_implicit(
    tfidf_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df, max_evals=300, seed=6789))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

**Task 6\*.** Implement an SVRRecommender by replacing the LinearRegression model with the SVR model (`from sklearn.svm import SVR`). Use separate features for users and items, not the dot product.

Tune params of the SVR model to obtain as good results as you can. 

To do tuning properly:

- divide the set into training, validation and test sets (randomly divide the dataset in proportions 60%-20%-20%),
- train the model with different sets of tunable parameters on the training set, 
- choose the best tunable params based on results on the validation set, 
- provide the final evaluation metrics on the test set for the best model obtained during tuning.

Recommended method of tuning: use hyperopt. Install the package using the following command: `pip install hyperopt`
    
Print the RMSE, MAPE and TRE on the test set. Use seed 6789.

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import MultiLabelBinarizer

    
class SVRRecommender(Recommender):
    """
    SVR recommender class.
    """

    def __init__(self, kernel='rbf', c=1.0, epsilon=0.1):
        """
        Initialize base recommender params and variables.
        """
        super().__init__()
        self.model = None
        self.mlb = None
        self.kernel = kernel
        self.c = c
        self.epsilon = epsilon
        self.mlb = None
        self.users_dict = None
        self.user_features = None
        self.uses_dot_product = False

    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.

        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id
            and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id
            and the item feature columns.
        """

        # Write your code here

    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns
        top n_recommendations for each user.

        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations
            should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations
            for each user.
        :rtype: pd.DataFrame
        """

        # Write your code here

    @staticmethod
    def _transform_genres(df):
        """
        Transforms a string with genres into a list of cleaned genre names.

        :param pd.DataFrame df: A DataFrame with 'genres' column.
        """
        df.loc[:, 'genres'] = df['genres'].str.replace("-", "_", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace(" ", "_", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace("(", "", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace(")", "", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.lower()
        df.loc[:, 'genres'] = df['genres'].str.split("|")
        return df
    
    
# Quick test of the recommender

svr_recommender = SVRRecommender()
svr_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = svr_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(HTML(recommendations.to_html()))

In [ ]:
svr_recommender = SVRRecommender(kernel='rbf', c=5.9672721141155, epsilon=0.8583733904374324)

results = [['SVRRecommender'] + list(evaluate_train_test_split_explicit(
    svr_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(results)

In [ ]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

# Split into train_validation and test sets

shuffle = np.arange(len(ml_ratings_df))
rng.shuffle(shuffle)
shuffle = list(shuffle)

train_test_split = 0.8
split_index = int(len(ml_ratings_df) * train_test_split)

train_validation = ml_ratings_df.iloc[shuffle[:split_index]].loc[:, ['user_id', 'item_id', 'rating']]
test = ml_ratings_df.iloc[shuffle[split_index:]].loc[:, ['user_id', 'item_id', 'rating']]

# Tune

def loss(tuned_params):
    svr_recommender = SVRRecommender(kernel='rbf', c=tuned_params['C'], epsilon=tuned_params['epsilon'])
    rmse, mre, tre = evaluate_train_test_split_explicit(
        svr_recommender, train_validation, ml_movies_df, seed=seed)
    return rmse

# Define the search space for hyperopt
# Write your code here


succeded = False
n_tries = 3
t = 0
while not succeded and t < n_tries:
    try:
        trials = Trials()
        best_param_set = fmin(loss, space=space, algo=tpe.suggest, max_evals=100, show_progressbar=True, trials=trials)
        succeded = True
    except:
        t += 1
        traceback.print_exc()

# Best params

print("C = {}".format(best_param_set['C']))
print("epsilon = {}".format(best_param_set['epsilon']))
    
# Test

C = 0.8078294657404328
epsilon = 0.3898720771476633

svr_recommender = SVRRecommender(C=best_param_set['C'], epsilon=best_param_set['epsilon'])

results = [['SVRRecommender'] + list(evaluate_train_test_split_explicit(
    svr_recommender, {'train': train_validation, 'test': test}, ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(HTML(results.to_html()))

**Task 7.** Gather the results for LinearRegressionRecommender, SVRRecommender, TFIDFRecommender, MostPopularRecommender, HighestRatedRecommender, RandomRecommender from the evaluate_train_test_split_implicit method and print them as a single DataFrame.

In [ ]:
# Write your code here